<a href="https://colab.research.google.com/github/Benjamindavid03/MachineLearningLab/blob/main/Spark_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare and Import the Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip "/content/drive/MyDrive/Colab Datasets/DelayedFlights.csv.zip"

Archive:  /content/drive/MyDrive/Colab Datasets/DelayedFlights.csv.zip
  inflating: DelayedFlights.csv      


# Setting up Pyspark in python

In [11]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [18]:
!wget -q https://downloads.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz

In [19]:
!tar xf spark-3.0.3-bin-hadoop2.7.tgz

In [20]:
!pip install -q findspark

In [24]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"
!export SPARK_HOME=/content/spark-3.0.3-bin-hadoop2.7
! echo $SPARK_HOME

/content/spark-3.0.3-bin-hadoop2.7


# Initialize Spark

In [25]:
import findspark
findspark.init()

# create a SparkSession, which is the entry point to Spark.



In [26]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [27]:
spark

# View Spark UI

In [30]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!ngrok config add-authtoken 2AyXjKIeBsO3IMCwnIGArgk0wPq_5JvTpU4uDDtEFwPh2EbAy
!curl -s http://localhost:4040/api/tunnels

--2022-06-23 11:16:18--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 52.202.168.65, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  13.19M  19.2MB/s    in 0.7s    

2022-06-23 11:16:19 (19.2 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ngrok                   
/bin/bash: ngrok: command not found
{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://aa3c-34-125-199-62.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":

In [32]:
df = spark.read.csv("DelayedFlights.csv", header=True, inferSchema=True) v #Load the CSV using spark

In [33]:
df.printSchema() #Show column details

root
 |-- _c0: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: double (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: double (nullable = true)
 |-- CRSElapsedTime: double (nullable = true)
 |-- AirTime: double (nullable = true)
 |-- ArrDelay: double (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: double (nullable = true)
 |-- TaxiOut: double (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted

In [35]:
df.show(5) # Show only 5 rows in DF

+---+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|_c0|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+---+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|  0|2008|    1|         3|        4| 2003.0|      1955| 2211.0|      2225|       

In [ ]:
df.count() #Number of rows in DF

In [37]:
df.select("Year","Month","DayOfMonth","FlightNum").show(5) #Display specific columns

+----+-----+----------+---------+
|Year|Month|DayOfMonth|FlightNum|
+----+-----+----------+---------+
|2008|    1|         3|      335|
|2008|    1|         3|     3231|
|2008|    1|         3|      448|
|2008|    1|         3|     3920|
|2008|    1|         3|      378|
+----+-----+----------+---------+
only showing top 5 rows



In [38]:
df.describe().show() # Describing the columns

+-------+------------------+-------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+-------------+------------------+-------+------------------+-----------------+------------------+-----------------+------------------+-------+-------+-----------------+-----------------+------------------+--------------------+----------------+--------------------+------------------+------------------+------------------+-------------------+------------------+
|summary|               _c0|   Year|             Month|        DayofMonth|         DayOfWeek|           DepTime|        CRSDepTime|          ArrTime|       CRSArrTime|UniqueCarrier|         FlightNum|TailNum| ActualElapsedTime|   CRSElapsedTime|           AirTime|         ArrDelay|          DepDelay| Origin|   Dest|         Distance|           TaxiIn|           TaxiOut|           Cancelled|CancellationCode|            Diverted|      CarrierDelay|      WeatherDelay|   

# Distinct values for Categorical columns

In [39]:
df.select("AirTime").distinct().show()

+-------+
|AirTime|
+-------+
|  305.0|
|  299.0|
|  496.0|
|  596.0|
|  558.0|
|  147.0|
|  170.0|
|  184.0|
|  576.0|
|  169.0|
|  160.0|
|  608.0|
|   70.0|
|   67.0|
|  311.0|
|  379.0|
|  486.0|
|    8.0|
|  571.0|
|  168.0|
+-------+
only showing top 20 rows



# Aggregate with Groupby

In [43]:
from pyspark.sql import functions as F
df.groupBy("Year").agg(F.sum("FlightNum")).show()

+----+--------------+
|Year|sum(FlightNum)|
+----+--------------+
|2008|    4230389297|
+----+--------------+



In [42]:
from pyspark.sql import functions as F
df.groupBy("Month").agg(F.sum("FlightNum")).show()

+-----+--------------+
|Month|sum(FlightNum)|
+-----+--------------+
|   12|     462827129|
|    1|     420956179|
|    6|     436885627|
|    3|     425587779|
|    5|     323301264|
|    9|     209158942|
|    4|     331227729|
|    8|     338770016|
|    7|     398696159|
|   10|     216036590|
|   11|     239755667|
|    2|     427186216|
+-----+--------------+



# Counting and Removing Null values

In [44]:
df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns]).show()

+---+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|_c0|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+---+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|  0|   0|    0|         0|        0|      0|         0|   7110|         0|       

In [48]:
df = df.fillna({'ActualElapsedTime':0, 'CRSElapsedTime':0})

In [49]:
df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns]).show()

+---+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|_c0|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+---+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|  0|   0|    0|         0|        0|      0|         0|   7110|         0|       

In [50]:
df.write.csv("/content/drive/My Drive/PySpark on Colab/preprocessed_data")


In [51]:
df.rdd.getNumPartitions()

2

In [ ]:
# Spark df to Pandas df
df_pd = df.toPandas()
# Store result
df_pd.to_csv("/content/drive/My Drive/PySpark on Colab/pandas_preprocessed_data.csv")

In [ ]:
import pandas as pd
test = pd.read_csv('/content/drive/My Drive/PySpark on Colab/pandas_preprocessed_data.csv')
test